In [4]:
# Rename magasine column
import json
import pandas as pd

filepath = 'articles/NEI_articles.json'
with open(filepath, 'r', encoding='utf-8') as file:
    data = json.load(file)

for article in data:
    article['magasine'] = 'Nuclear Engineering International'

with open(filepath, 'w', encoding='utf-8') as file:
    json.dump(data, file, ensure_ascii=False, indent=4)


In [ ]:
# Data exploration, check consistency etc.
df = pd.read_json('NCE_articles.json')
a = df['author'].unique()
for e in a:
    print(e)


In [5]:
# Merging articles
import glob
import json

json_files = glob.glob('articles/*.json')

all_articles = []

for filepath in json_files:
    with open(filepath, 'r', encoding='utf-8') as file:
        data = json.load(file)
        all_articles.extend(data)

with open('articles/all_articles.json', 'w', encoding='utf-8') as file:
    json.dump(all_articles, file, ensure_ascii=False, indent=4)